In [40]:
#Libreria para manejo del sistema de archivos
import os

#Libreria para crear/leer/cambiar archivos
import codecs

#Libreria para manejo de archivos json
import json

#Libreria para creacion de numeros aleatorios
import random

#Libreria para manejo de patrones de texto/archivos
import glob

#Libreria para manejo de archivos "parquet"
import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq

#Libreria para el manejo de datos
import pandas as pd

#Libreria para el manejo de elementos matematicos
import numpy as np



# Librerías para graficar
import matplotlib.pyplot as plt
%matplotlib inline


In [41]:
import re 
import nltk 

nltk.download("stopwords") 
from nltk.corpus import stopwords 

from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\matts\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Tool recommender through content processing (NLP)**

### **1. Pre-procesamiento de los datos**

In [42]:
cvs = pd.read_csv("my_data.csv")
cvs

,raw_text,status
0,DIEGO RODOLFO HERRER A RUBIO \nDIEGO RODOLFO...,considerados
1,ricardo herrera\njacobo\nIngeniero Energético ...,considerados
2,\n \n \n \nPERFIL \nEgresado pasante de rela...,considerados
3,\n Herman Gregorio Parra Torcatt \n\n \n \n...,considerados
4,05/Octubre /2022 \n \n \nJuan Jorge Aparicio...,considerados
5,LUIS ANGEL GRIMALDO\nCERRITOS\nTSU INGENIERIA ...,considerados
6,\n \n\n \n \n \n\nAFICIONES \nMúsica \nFut...,no_considerados
7,\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...,no_considerados
8,Nelson Salazar\nIngeniero Electrico\nJerez de ...,no_considerados


In [43]:
bow = cvs.copy()

In [44]:
# Codificamos las categorías
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
bow["status"] = le.fit_transform(bow["status"])

In [45]:
# Para conocer como fue el encoding
le = LabelEncoder()
le.fit_transform(cvs["status"])
keys = le.classes_
values = le.transform(le.classes_)
dictionary = dict(zip(keys, values))
dictionary

{'considerados': 0, 'no_considerados': 1}

**Limpiamos los textosv(obtenemos el corpus)**

In [46]:
corpus = [] 
for i in list(bow.loc[:,"raw_text"]):
    # 1. Quitamos todas las letras:
    review = re.sub("[^a-zA-Z]", " ", i) 
    # 2. Todo en minúsculas:
    review = review.lower() 
    # 3. Separamos oraciones para obtener puras palabras
    review = review.split()
    
    # 4. Stemming (a veces vale la pena considerar esto)
    # Queremos quitar palabras irrelevantes como artículos
    ps = PorterStemmer()
    all_stopwords = stopwords.words("spanish") 
    #all_stopwords.remove("palabra") # --> para excepciones
    review = [ps.stem(word) for word in review if word not in set(all_stopwords)]
    
    # Unir oraciones con espacis entre cada palabra
    review = " ".join(review)
    corpus.append(review)

In [47]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus).toarray()
y = bow["status"].values

In [48]:
len(X[0])

1025

In [49]:
# Train y test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### **2. Construcción de Red Neuronal**

# **Properly build and evaluate an ANN for multiclass classification**

https://michael-fuchs-python.netlify.app/2021/02/23/nn-artificial-neural-network-for-multi-class-classfication/


Consideraciones:

https://www.linkedin.com/pulse/choosing-number-hidden-layers-neurons-neural-networks-sachdev/

- Número de hidden layers = 1-2 (datasets pequeños), 3-5 (datasests largos)
- Número de nodos: sqrt(input layer nodes * output layer nodes)


In [50]:
np.sqrt(len(X[0])*bow["status"].nunique())

45.27692569068709

Tenemos muchas columnas pero no muchas filas así que 3 puede ser un buen número de hidden layers y 311 la cantidad de neuronas que debemos tener en el siguiente orden por layer:
- 1 hidden layer: 311*3/4
- 2 hidden layer: 311*2/4
- 3 hidden layer: 311*1/4
- output layer: cantidad de categorías

In [51]:
import tensorflow as tf

In [52]:
ann = tf.keras.models.Sequential()

In [53]:
ann.add(tf.keras.layers.Dense(units=233, activation="relu"))

In [54]:
ann.add(tf.keras.layers.Dense(units=155, activation="relu")) 

In [55]:
ann.add(tf.keras.layers.Dense(units=77, activation="relu")) 

In [56]:
ann.add(tf.keras.layers.Dense(units=1, activation="softmax"))

In [57]:
ann

### **3. Entrenamiento de Red Neuronal**

**Considerations**

For target variable y:
- binary: loss=binary_crossentropy
- multicategorical (one-hot-encoded y): categorical_crossentropy 
- multicategorical (integer y): sparse_categorical_crossentropy

In [58]:
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [60]:
ann.fit(x=X, y=y, batch_size=32, epochs=300)

Epoch 1/300
1/1 [==============================] - 0s 10ms/step - loss: 4.0820e-06 - accuracy: 0.3333
Epoch 2/300
1/1 [==============================] - 0s 9ms/step - loss: 4.0730e-06 - accuracy: 0.3333
Epoch 3/300
1/1 [==============================] - 0s 8ms/step - loss: 4.0642e-06 - accuracy: 0.3333
Epoch 4/300
1/1 [==============================] - 0s 7ms/step - loss: 4.0556e-06 - accuracy: 0.3333
Epoch 5/300
1/1 [==============================] - 0s 7ms/step - loss: 4.0472e-06 - accuracy: 0.3333
Epoch 6/300
1/1 [==============================] - 0s 9ms/step - loss: 4.0390e-06 - accuracy: 0.3333
Epoch 7/300
1/1 [==============================] - 0s 8ms/step - loss: 4.0309e-06 - accuracy: 0.3333
Epoch 8/300
1/1 [==============================] - 0s 8ms/step - loss: 4.0230e-06 - accuracy: 0.3333
Epoch 9/300
1/1 [==============================] - 0s 8ms/step - loss: 4.0152e-06 - accuracy: 0.3333
Epoch 10/300
1/1 [==============================] - 0s 8ms/step - loss: 4.0075e-06 - accur